In [120]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

In [102]:
x = np.load("brain_imaging/train_X.npy")

In [103]:
y = np.load("brain_imaging/train_binary_Y.npy")

In [104]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [105]:
x_train /= 255
x_test /= 255

In [106]:
model = Sequential()
model.add(Convolution2D(32, kernel_size=(3, 3),padding='same',input_shape=x_train[0].shape))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train[0].shape[0]))
model.add(Activation('sigmoid'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by InteractiveShellApp.init_path()


In [107]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [108]:
check = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.5f}.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')
model.fit(x_train, y_train, batch_size=32, nb_epoch=20,callbacks=[check],validation_data=(x_test,y_test))


C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3681 samples, validate on 921 samples
Epoch 1/20
3681/3681 [==============================] - 74s 20ms/step - loss: 0.3190 - acc: 0.8871 - val_loss: 0.2844 - val_acc: 0.8954
Epoch 2/20
3681/3681 [==============================] - 56s 15ms/step - loss: 0.2384 - acc: 0.9122 - val_loss: 0.2032 - val_acc: 0.9253
Epoch 3/20
3681/3681 [==============================] - 55s 15ms/step - loss: 0.1953 - acc: 0.9265 - val_loss: 0.1815 - val_acc: 0.9301
Epoch 4/20
3681/3681 [==============================] - 55s 15ms/step - loss: 0.1742 - acc: 0.9313 - val_loss: 0.1613 - val_acc: 0.9346
Epoch 5/20
3681/3681 [==============================] - 55s 15ms/step - loss: 0.1539 - acc: 0.9400 - val_loss: 0.1365 - val_acc: 0.9449
Epoch 6/20
3681/3681 [==============================] - 55s 15ms/step - loss: 0.1331 - acc: 0.9476 - val_loss: 0.1500 - val_acc: 0.9439
Epoch 7/20
3681/3681 [==============================] - 56s 15ms/step - loss: 0.1244 - acc: 0.9514 - val_loss: 0.1178 - val_acc: 0.9542
E

In [109]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 26, 31, 32)        6656      
_________________________________________________________________
activation_71 (Activation)   (None, 26, 31, 32)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 24, 29, 64)        18496     
_________________________________________________________________
activation_72 (Activation)   (None, 24, 29, 64)        0         
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 12, 14, 64)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 12, 14, 64)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 12, 14, 64)        36928     
__________

In [134]:
xt = x_test
yt = y_test
out = model.predict_proba(xt)
out = np.array(out)

threshold = np.arange(0.1,0.9,0.1)

acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(yt[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best thresholds", best_threshold)
y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(yt.shape[1])] for i in range(len(yt))])

print("-"*40)
print("Matthews Correlation Coefficient")
print("Class wise accuracies")
print(accuracies)


best thresholds [ 0.3  0.2  0.1  0.3  0.2  0.7  0.8  0.7  0.5  0.5  0.4  0.3  0.6  0.1  0.6
  0.5  0.4  0.4  0.1]
----------------------------------------
Matthews Correlation Coefficient
Class wise accuracies
[0.68609099572233068, 0.90799200799200797, 0.61535915596634483, 0.87903299939729562, 0.85508994597996535, 0.80523162479109023, 0.73364300541622374, 0.8573944957309596, 0.811870370371365, 0.62376196351752422, 0.80205631821048196, 0.75077315602288541, 0.86742905497661404, 0.93490249201948594, 0.76797909397530206, 0.95642586457477863, 0.74361219250327459, 0.93960655737704923, 0.55051538965813551]


In [147]:
valid_x_test = np.load("brain_imaging/valid_test_X.npy")

In [149]:
pred_prob = model.predict_proba(valid_x_test)

In [151]:
pred_prob

array([[ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)